# Cumulative sums
The research question for this notebook is: "What parameters can we best use to say something about directionality?"

We sort the bins based on how full they are. The y-axis will be the cumulative sum of this data fraction. The x-axis will be the percentage of the free volume in which this data is. 

The hypothesis is that this will be a straight line for non-directional interaction between pairs and would be steeper for more directional interactions.

In [ ]:
import sys

sys.path.append('..//scripts//')

from helpers.density_helpers import find_available_volume, prepare_df
from classes.Settings import Settings, Radii

from constants.paths import WORKDIR, RADII_CSV

central_groups = ["ArCI", "REt", "RNO2", "RCOMe", "NO3", "RC6F5", "RC6H5", "H2O"]
contact_groups = ["ArCH", "C2CH2", "CCH3", "CF", "R2CO", "RC6H5", "RCN", "XH", "XH"]
to_count =       ["H",    "H",      "H",   "F", "O",     "centroid", "N", "H", "O"]

In [ ]:
%matplotlib notebook
import pandas as pd

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

import scipy as sp
from scipy import optimize

import numpy as np

In [ ]:
# make volume csv if not exists
import os
import csv

tolerance = 0.5

if not os.path.exists('../../results/volumes_free.csv'):
    with open('../../results/volumes_free.csv', 'w', newline='') as freevolumesfile:
        csvwriter = csv.writer(freevolumesfile)
        csvwriter.writerow(['central', 'contact', 'to_count', 'volume'])

        for central_group in central_groups:   
            for to_count_contact, contact_group in zip(to_count, contact_groups):              
                datafile = "..\\data\\" + central_group + "\\" + central_group + "_" + contact_group + "_vdw.5.cor"

                settings = Settings('..\\..', datafile)
                settings.set_atom_to_count(to_count_contact)

                ############ find out volume of the central group 
                aligned_df = pd.read_csv(settings.get_aligned_csv_filename(), header=0)
                avg_fragment = pd.read_csv(settings.get_avg_frag_filename())

                contact_group_radius = Radii(RADII_CSV).get_vdw_distance_contact(aligned_df, settings)
                volume = find_available_volume(avg_fragment=avg_fragment, extra=(tolerance + contact_group_radius))
                
                csvwriter.writerow([central_group, contact_group, to_count_contact, round(volume, 4)])
                
                print(f"Free Volume {central_group}, {contact_group} ({to_count_contact}) is {volume :.2f}")
                
                
if not os.path.exists('../../results/volumes_total.csv'):
    with open('../../results/volumes_total.csv', 'w', newline='') as freevolumesfile:
        csvwriter = csv.writer(freevolumesfile)
        csvwriter.writerow(['central', 'contact', 'to_count', 'volume'])

        for central_group in central_groups:   
            for to_count_contact, contact_group in zip(to_count, contact_groups):
           
                datafile = "..\\data\\" + central_group + "\\" + central_group + "_" + contact_group + "_vdw.5.cor"

                settings = Settings('..\\..', datafile)
                settings.set_atom_to_count(to_count_contact)

                ############ find out volume of the central group 
                aligned_df = pd.read_csv(settings.get_aligned_csv_filename(), header=0)
                avg_fragment = pd.read_csv(settings.get_avg_frag_filename())

                contact_group_radius = Radii(RADII_CSV).get_vdw_distance_contact(aligned_df, settings)
                volume = find_available_volume(avg_fragment=avg_fragment, extra=(tolerance + contact_group_radius), total=True)
                
                csvwriter.writerow([central_group, contact_group, to_count_contact, round(volume, 4)])
                
                print(f"Total Volume {central_group}, {contact_group} ({to_count_contact}) is {volume :.2f}")

In [ ]:
volumes_free = pd.read_csv('../../results/volumes_free.csv')
volumes_total = pd.read_csv('../../results/volumes_total.csv')

display(volumes_free)
display(volumes_total)

In [ ]:
def get_data(central_group, contact_group, to_count_contact, resolution):
    datafile = "..\\data\\" + central_group + "\\" + central_group + "_" + contact_group + "_vdw.5.cor"

    settings = Settings('..\\..', datafile)
    settings.set_atom_to_count(to_count_contact)
    settings.set_resolution(resolution)

    hdf_file = "../../results/pairs/" + central_group + "/" + central_group + '_' + contact_group + '_vdw.5/' + central_group + "_" + contact_group + "_density.hdf"
    density_df = pd.read_hdf(settings.get_density_df_filename(), settings.get_density_df_key())

    density_df['datafrac_normalized'] = density_df[to_count_contact] / density_df[to_count_contact].sum()
    
    # sort from max bin to min bin
    density_df = density_df.sort_values(by=to_count_contact, ascending=False).reset_index()
    
    density_df['cum_data'] = density_df["datafrac_normalized"].cumsum()
    
    density_df['cum_data_not_normalized'] = density_df[to_count_contact].cumsum()
    density_df['cum_vol'] = density_df.index * resolution**3
    
    return density_df

In [ ]:
def normalize_cum_vol(density_df, volumes):
    free_volume = volumes.loc[(volumes.central == central_group) & (volumes.contact == contact_group) & (volumes.to_count == to_count_contact), 'volume'].item()  
    free_volume = (free_volume / 2) + 0.1 * free_volume 
    
    assert density_df.cum_data.max() > 0.99 and density_df.cum_data.max() < 1.01, "Cumulative volume should be 1"
    density_df = density_df[density_df['cum_vol'] <= free_volume].copy()
        
    density_df['cum_vol'] = density_df['cum_vol'] / free_volume
    
    return density_df


def make_plot(density_df):    
    cluster_frac = 0.25
    threshold = density_df.datafrac_normalized.max() * cluster_frac
    in_cluster = density_df[density_df.datafrac_normalized >= threshold]
            
    ax = plt.gca()
    color = next(ax._get_lines.prop_cycler)['color']
    
    plt.plot(density_df['cum_vol'], density_df['cum_data'], color=color, label=f"{central_group}--{contact_group} ({to_count_contact})")
    plt.scatter(in_cluster['cum_vol'], in_cluster['cum_data'], color=color)

In [ ]:
resolutions = [0.25, 0.5, 0.75]

for resolution in resolutions:
    for central_group in central_groups:
        plt.figure(figsize=(9, 5))
        plt.title(f"Cumulative datafraction {central_group}\nResolution {resolution}")

        for to_count_contact, contact_group in zip(to_count, contact_groups):
            if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
                continue
                
            density_df = get_data(central_group, contact_group, to_count_contact, resolution)
            density_df = normalize_cum_vol(density_df, volumes_total)
            make_plot(density_df)

        plt.ylabel("Cumulative datafraction inside volume central group")
        plt.xlabel("Cumulative fraction of total volume central group")

        plt.legend(loc="lower right")
        plt.savefig(f'../../results/cumsums/per_central_group_total_volume/cumsum_data_and_volume_{central_group}_res{resolution: .2f}.png')
        plt.show()

In [ ]:
resolution = 0.25

for to_count_contact, contact_group in zip(to_count, contact_groups):
    plt.figure(figsize=(9, 5))
    plt.title("Cumulative datafraction, contact group: " + contact_group)

    for central_group in central_groups:
        if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
            continue
                
        density_df = get_data(central_group, contact_group, to_count_contact, resolution)
        density_df = normalize_cum_vol(density_df, volumes_total)
        make_plot(density_df)

        plt.ylabel("Cumulative datafraction inside volume central group")
        plt.xlabel("Cumulative fraction of total volume central group")
    
    plt.legend(loc="lower right")
    plt.savefig(f'../../results/cumsums/per_contact_group_total_volume/cumsum_data_and_volume_per_contact_{contact_group}_res{resolution}.png')

# Analyzing the plots
We see that some of the lines do not get to to the datafraction of 1.0. What is going on there?

This is the data that does not lie within the central volume of the central group; for example, XH (H) can interact with O. We do not throw away the H's that are too far away (further than vdwH, vdwClosestAtomCentralGroup + 0.5), which means that the H's that are next to the interacting O's, sometimes are not in the central volume.

Is there a way to show how we got the directionality in these plots?


# How to compare directionality-sets?

Heatmaps!

In [ ]:
df_dir = pd.DataFrame(index=central_groups, columns=[x+"-"+y for x,y in zip(contact_groups, to_count)])
df_dir

In [ ]:
def calc_all_directionalities(cluster_frac, resolution, volumes):
    directionalities = []
    
    for to_count_contact, contact_group in zip(to_count, contact_groups):
        for central_group in central_groups:
            if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
                continue
    
            datafile = "..\\data\\" + central_group + "\\" + central_group + "_" + contact_group + "_vdw.5.cor"

            settings = Settings('..\..', datafile)
            settings.set_atom_to_count(to_count_contact)
            settings.set_resolution(resolution)

            hdf_file = "../../results/pairs/" + central_group + "/" + central_group + '_' + contact_group + '_vdw.5/' + central_group + "_" + contact_group + "_density.hdf"
            density_df = pd.read_hdf(hdf_file, settings.get_density_df_key())
#             density_df = pd.read_hdf(settings.get_density_df_filename(), settings.get_density_df_key())

            density_df['datafrac_normalized'] = density_df[to_count_contact] / density_df[to_count_contact].sum()
            
            threshold = density_df.datafrac_normalized.max() * cluster_frac
            in_cluster = density_df[density_df.datafrac_normalized >= threshold]
            
            Vavailable = volumes.loc[(volumes.central == central_group) & (volumes.contact == contact_group) & (volumes.to_count == to_count_contact), 'volume'].item()  

            datafrac = in_cluster.datafrac_normalized.sum()
            Vcluster = len(in_cluster) * resolution**3
            
            directionality = datafrac / Vcluster * Vavailable
            directionalities.append(directionality)
            
    return directionalities


In [ ]:
def fill_df(directionalities):
    df = pd.DataFrame(index=central_groups, columns=[x+"-"+y for x,y in zip(contact_groups, to_count)])
    
    i = -1
    for to_count_contact, contact_group in zip(to_count, contact_groups):
        for central_group in central_groups:
            if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
                continue
            else:
                i+=1

            column = contact_group + "-" + to_count_contact
            df.loc[df.index == central_group, column] = float(directionalities[i])
            
    return df.astype("float")

In [ ]:
def make_heatmap(df_dir, title):
    fig, ax = plt.subplots(figsize=(8,4))
    plt.title(title)
    sns.heatmap(df_dir, cmap=sns.diverging_palette(145, 300, s=60, as_cmap=True), annot = True, fmt='.3g', linewidth =0.1)

    plt.xlabel("Contact group")
    plt.xticks(rotation=30)
    
    plt.ylabel("Central group")
    plt.yticks(rotation=0)

    fig.subplots_adjust(bottom=0.3)
    plt.show()

cluster_frac = 0.25

resolution = 0.25
directionalities = calc_all_directionalities(cluster_frac=0.25, resolution=0.25, volumes=volumes_total)
df = fill_df(directionalities)
make_heatmap(df, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')

resolution = 0.5
directionalities_res05 = calc_all_directionalities(cluster_frac=0.25, resolution=0.5, volumes=volumes_total)
df_res05 = fill_df(directionalities_res05)
make_heatmap(df_res05, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')

resolution = 0.75
directionalities_res075 = calc_all_directionalities(cluster_frac=0.25, resolution=0.75, volumes=volumes_total)
df_res075 = fill_df(directionalities_res075)
make_heatmap(df_res075, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')

In [ ]:
resolution = 0.5

cluster_frac=0.25
directionalities_res05 = calc_all_directionalities(cluster_frac=0.25, resolution=0.5, volumes=volumes_total)
df_res05 = fill_df(directionalities_res05)
make_heatmap(df_res05, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')

cluster_frac = 0.50
directionalities_res05_kfrac05 = calc_all_directionalities(cluster_frac=0.50, resolution=0.5, volumes=volumes_total)
df_res05_kfrac05 = fill_df(directionalities_res05_kfrac05)
make_heatmap(df_res05_kfrac05, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')

cluster_frac = 0.75
directionalities_res05_kfrac075 = calc_all_directionalities(cluster_frac=0.75, resolution=0.5, volumes=volumes_total)
df_res05_kfrac075 = fill_df(directionalities_res05_kfrac075)
make_heatmap(df_res05_kfrac075, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')

In [ ]:
resolution = 0.25

cluster_Frac = 0.25
directionalities_res025 = calc_all_directionalities(cluster_frac=0.25, resolution=0.25, volumes=volumes_total)
df_res025 = fill_df(directionalities_res025)
make_heatmap(df_res025, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')
    
cluster_frac = 0.50
directionalities_res025_kfrac05 = calc_all_directionalities(cluster_frac=0.50, resolution=0.25, volumes=volumes_total)
df_res025_kfrac05 = fill_df(directionalities_res025_kfrac05)
make_heatmap(df_res025_kfrac05, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')

cluster_frac = 0.75
directionalities_res025_kfrac075 = calc_all_directionalities(cluster_frac=0.75, resolution=0.25, volumes=volumes_total)
df_res025_kfrac075 = fill_df(directionalities_res025_kfrac075)
make_heatmap(df_res025_kfrac075, f"Directionalities, resolution {resolution :.2f}\nBin threshold: {cluster_frac :.2f}")
plt.savefig(f'../../results/cumsums/directionalities_formula_res{resolution:.2f}_bin_threshold{cluster_frac :.2f}.png')

In [ ]:
def exp_func(x, a, b):
    return a * np.exp(b * x)

def inv_exp_func(x, a, b):
    return a * (1 - np.exp(b * -x))

In [ ]:
df_popt = pd.DataFrame(index=central_groups, columns=[x+"-"+y for x,y in zip(contact_groups, to_count)])
df_rsquared = pd.DataFrame(index=central_groups, columns=[x+"-"+y for x,y in zip(contact_groups, to_count)])

In [ ]:
resolution = 0.5

for central_group in central_groups:
    plt.figure(figsize=(9, 4))
    plt.title(central_group)
    
    for to_count_contact, contact_group in zip(to_count, contact_groups):
        
        if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
            continue
        
        density_df = get_data(central_group, contact_group, to_count_contact, resolution)        
        density_df = normalize_cum_vol(density_df, volumes_total)
        
        x = density_df['cum_vol']
        y = density_df['cum_data']
        
        popt, _  = sp.optimize.curve_fit(inv_exp_func, x, y)

        ax = plt.gca()
        color = next(ax._get_lines.prop_cycler)['color']
        
        plt.plot(x, y, linestyle='dotted', color=color)
        plt.ylabel("Cumulative datafraction")
        plt.xlabel("Cumulative fraction of free volume")
        
        # save b in df
        column = contact_group + "-" + to_count_contact
        df_popt.loc[df_popt.index == central_group, column] = popt[1]
        
        # calculate r squared and save that as well
        residuals = y - inv_exp_func(x, *popt)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        r_squared = 1 - (ss_res / ss_tot)
        
        df_rsquared.loc[df_rsquared.index == central_group, column] = r_squared
        
        label = f"{contact_group}({to_count_contact}) {round(popt[0], 2)}(1-e^(-{round(popt[1], 2)}x)"
        plt.plot(x, inv_exp_func(x, *popt), label=label, color=color)
    
    plt.legend()
    plt.show()

In [ ]:
def R_bold_condition(val):
    bold = 'bold' if val < 0.95 else ''

    return 'font-weight: %s' % bold

def R_yellow_condition(val):
    color = 'yellow' if val < 0.95 else ''
    return 'background-color:' + color

display(df_popt)

print("Rsquared values")
display(df_rsquared.style.applymap(R_bold_condition).applymap(R_yellow_condition))



In [ ]:
make_heatmap(df_popt.astype('float'), "b in fitted inverse exponential $a(1-e^{-bx})$")
plt.savefig('../../results/cumsums/fitted_inv_exp_res0.5.png')

# Now the same with a log function

In [ ]:
df_popt_log = pd.DataFrame(index=central_groups, columns=[x+"-"+y for x,y in zip(contact_groups, to_count)])
df_rsquared_log = pd.DataFrame(index=central_groups, columns=[x+"-"+y for x,y in zip(contact_groups, to_count)])

In [ ]:
def log_func(x, a, b, c):
    return a + b * np.log(c * x)

resolution = 0.25

for central_group in central_groups:
    plt.figure(figsize=(9, 5))
    plt.title(central_group)
    
    for to_count_contact, contact_group in zip(to_count, contact_groups):
        
        if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
            continue
        
        density_df = get_data(central_group, contact_group, to_count_contact, resolution)        
        density_df = normalize_cum_vol(density_df, volumes_total)
        
        x=density_df['cum_vol']
        y=density_df['cum_data']
        
        popt, _  = sp.optimize.curve_fit(log_func, x[1:], y[1:])

        ax = plt.gca()
        color = next(ax._get_lines.prop_cycler)['color']
        
        plt.plot(x, y, linestyle='dotted', color=color)
        plt.ylabel("Amount of points in bins")
        plt.xlabel("Bin volume")
        
        # save c in df
        column = contact_group + "-" + to_count_contact
        df_popt_log.loc[df_popt.index == central_group, column] = popt[2]
        
        # calculate r squared and save that as well
        residuals = y - log_func(x[1:], *popt)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y[1:] - np.mean(y[1:]))**2)
        r_squared = 1 - (ss_res / ss_tot)
        
        df_rsquared_log.loc[df_rsquared.index == central_group, column] = r_squared
        
        label = f"{contact_group}({to_count_contact}) {round(popt[0], 2)} + {round(popt[1], 2)} log({round(popt[2], 2)}x)"
        plt.plot(x[1:], log_func(x[1:], *popt), label=label, color=color)
    
    plt.legend()
    plt.show()

In [ ]:
display(df_popt_log)

print("Rsquared values")
display(df_rsquared_log.style.applymap(R_bold_condition).applymap(R_yellow_condition))

In [ ]:
make_heatmap(df_popt_log.astype('float'), "c in fitted log $a + b \log(c*x)$")
plt.savefig('../../results/cumsums/fitted_log.png')

In [ ]:
def ln_func(x, a, b, c):
    return a + b*np.log(c*x)

resolution = 0.5

# central_groups = ["NO3"]
# contact_groups = ["RC6H5"]
# to_count = ["centroid"]

for central_group in central_groups:
    plt.figure(figsize=(10, 9))
    plt.title(central_group)
    
    for to_count_contact, contact_group in zip(to_count, contact_groups):
        
        if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
            continue
        
        density_df = get_data(central_group, contact_group, to_count_contact, resolution)        
        density_df = normalize_cum_vol(density_df, volumes_total)
        
        x=density_df['cum_vol']
        y=density_df['cum_data']
        
        popt2, _  = sp.optimize.curve_fit(ln_func, x[1:], y[1:])
        popt3, _  = sp.optimize.curve_fit(inv_exp_func, x, y)

        ax = plt.gca()
        color = next(ax._get_lines.prop_cycler)['color']
        
        plt.scatter(x, y, s=0.2, color=color)
        plt.ylabel("Cumulative datafraction")
        plt.xlabel("Cumulative fraction of free volume")
                
        residuals = y[1:] - ln_func(x[1:], *popt2)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y[1:] - np.mean(y[1:]))**2)
        r_squared = 1 - (ss_res / ss_tot)
        
        print(f"R squared ln {r_squared}")
                
        # calculate r squared and save that as well
        residuals = y - inv_exp_func(x, *popt3)
        ss_res = np.sum(residuals**2)
        ss_tot = np.sum((y - np.mean(y))**2)
        r_squared = 1 - (ss_res / ss_tot)
        
        print(f"R squared inv exp {r_squared}")
        
        label = f"{contact_group}({to_count_contact}) {round(popt[0], 2)}(1-e^(-{round(popt[1], 2)}x)"
        plt.plot(x, inv_exp_func(x, *popt3), label=label, color=color)
                  
        label = f"{contact_group}({to_count_contact}) {round(popt2[0], 2)} + {round(popt2[1], 2)} ln({round(popt2[2], 2)}x)"
        plt.plot(x[1:], ln_func(x[1:], *popt2), label=label, color=color)
    
    plt.legend()
    plt.show()
#         
        


# test

kijk op bv 0.9 en dan abs volume bins / free volume of total volume

In [ ]:
resolution = 0.25

central_groups = ["ArCI"]

for central_group in central_groups:
    plt.figure(figsize=(9, 4))
    plt.title(central_group)
    
    for to_count_contact, contact_group in zip(to_count, contact_groups):
        
        if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
            continue
        
        density_df = get_data(central_group, contact_group, to_count_contact, resolution)        
       
        density_df = density_df[density_df.datafrac_normalized > 0]

        x = range(len(density_df))
        max_ = x[-1]
        x = [n/max_ for n in x]
        y = density_df['cum_data']
        
        label = f"{contact_group}({to_count_contact})"
#         plt.xlim(0,0.2)
#         plt.ylim(0,0.2)
        plt.plot(x, y, label=label)
    
    plt.legend()
    plt.show()

In [ ]:
resolution = 0.25

central_groups = ["ArCI"]

for central_group in central_groups:
    plt.figure(figsize=(9, 4))
    plt.title(central_group)
    
    for to_count_contact, contact_group in zip(to_count, contact_groups):
        
        if central_group == "RC6H5" and (contact_group == "RC6H5" or contact_group == "ArCH"):
            continue
        
        density_df = get_data(central_group, contact_group, to_count_contact, resolution)        
       

        density_df = density_df[density_df.datafrac_normalized > 0]
        
        min_ = density_df.datafrac_normalized.min()
        print(min_)
        
        density_df = density_df[density_df.datafrac_normalized > min_]

        x = range(len(density_df))
        y = density_df["datafrac_normalized"]
        label = f"{contact_group}({to_count_contact})"
#         plt.xlim(0,0.2)
#         plt.ylim(0,0.2)
        plt.plot(x, y, label=label)
    
    plt.legend()
    plt.show()